In [ ]:
!pip install transformers datasets evaluate rouge_score
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 91.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.0 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=2d6f11b7c3a8f37b4bbf2065e69a442e542bd7c844cec2cc1d582498c96c4948
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8

In [ ]:
from datasets import load_dataset

billsum = load_dataset("cnn_dailymail", "3.0.0", split="train")

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
billsum = billsum.train_test_split(test_size=0.1)

In [ ]:
from transformers import AutoTokenizer

checkpoint = "t5-small"
# checkpoint = "sshleifer/distilbart-cnn-12-6"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
prefix = "summarize: "


def preprocess_function(examples):
    """
    Preprocesses examples for input to the sequence-to-sequence model.

    Args:
        examples (dict): A dictionary containing input article and corresponding highlights.

    Returns:
        dict: A dictionary containing model inputs including tokenized and preprocessed data.

    This function takes a dictionary of examples, where each example contains an "article" and its corresponding
    "highlights". It preprocesses the data by tokenizing the input articles, adding a prefix, and truncating if
    necessary. It also tokenizes the target highlights and attaches them as "labels". The preprocessed data is then
    returned in a dictionary format suitable for input to the sequence-to-sequence model.
    """
    # Tokenize input articles with the added prefix and truncate if necessary
    inputs = [prefix + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    # Tokenize target highlights and attach them as "labels"
    labels = tokenizer(text_target=examples["highlights"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_billsum = billsum.map(preprocess_function, batched=True)

Map:   0%|          | 0/10341 [00:00<?, ? examples/s]

Map:   0%|          | 0/1149 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [ ]:
import evaluate

rouge = evaluate.load("rouge")


In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    """
    Computes evaluation metrics for generated summaries compared to reference summaries.

    Args:
        eval_pred (tuple): A tuple containing model-generated predictions and corresponding labels.

    Returns:
        dict: A dictionary containing computed evaluation metrics, including ROUGE scores and average generation length.

    This function takes a tuple of model-generated predictions and corresponding labels. It decodes the predictions and
    labels, computes ROUGE scores by comparing the generated summaries with the reference summaries, and calculates the
    average generation length. The computed evaluation metrics are returned in a dictionary format with rounded values.
    """

    predictions, labels = eval_pred

    # Decode predictions and labels to text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Compute ROUGE scores using decoded predictions and labels
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    # Calculate average generation length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [ ]:
tokenized_billsum_train = tokenized_billsum['train'].shuffle(seed=42).select(range(100000))
tokenized_billsum_test = tokenized_billsum['test'].shuffle(seed=42).select(range(10000))
tokenized_billsum_train

Dataset({
    features: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 100000
})

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="my_awesome_billsum_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_billsum_train,
    eval_dataset=tokenized_billsum_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.869700,1.678559,0.239600,0.111000,0.199000,0.199000,18.977900
2,1.856700,1.673672,0.239200,0.110300,0.198200,0.198300,18.980500


TrainOutput(global_step=12500, training_loss=1.8739453515625, metrics={'train_runtime': 2939.0943, 'train_samples_per_second': 68.048, 'train_steps_per_second': 4.253, 'total_flos': 5.41367205888e+16, 'train_loss': 1.8739453515625, 'epoch': 2.0})

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
PATH = "gdrive/MyDrive/Saved_Models/"

Mounted at /content/gdrive


In [ ]:
trainer.save_model(PATH)

In [ ]:
# Repeating everything to run for the saved model

!pip install transformers datasets evaluate rouge_score
!pip install accelerate -U

from datasets import load_dataset

# billsum = load_dataset("billsum", split="ca_test")
billsum = load_dataset("cnn_dailymail", "3.0.0", split="train")

billsum = billsum.train_test_split(test_size=0.1)

billsum2 = load_dataset("cnn_dailymail", "3.0.0", split="test")

billsum2 = billsum2.train_test_split(test_size=0.9999)


from transformers import AutoTokenizer
from google.colab import drive
drive.mount('/content/gdrive')
PATH = "gdrive/MyDrive/Saved_Models/"

checkpoint = PATH

# checkpoint = "t5-small"
# checkpoint = "sshleifer/distilbart-cnn-12-6"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

prefix = "summarize: "
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["highlights"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_billsum = billsum.map(preprocess_function, batched=True)
tokenized_billsum2 = billsum2.map(preprocess_function, batched=True)


from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

import evaluate

rouge = evaluate.load("rouge")

import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer


model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.8 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=69aebaa1f083f58b05b88013b2870dce8b8747dae74be853940486691a3e8aa8
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Mounted at /content/gdrive


Map:   0%|          | 0/258401 [00:00<?, ? examples/s]

Map:   0%|          | 0/28712 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/11489 [00:00<?, ? examples/s]

In [ ]:
tokenized_billsum_train = tokenized_billsum['train'].shuffle().select(range(250000))
tokenized_billsum_test = tokenized_billsum2['test'].shuffle().select(range(11000))
# tokenized_billsum_train

training_args = Seq2SeqTrainingArguments(
    output_dir="my_awesome_billsum_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    # push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    # train_dataset=tokenized_billsum["train"],
    # eval_dataset=tokenized_billsum["test"],
    train_dataset=tokenized_billsum_train,
    eval_dataset=tokenized_billsum_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.827800,1.680420,0.247300,0.118000,0.204600,0.204600,18.998500


TrainOutput(global_step=15625, training_loss=1.8378835439453125, metrics={'train_runtime': 5007.848, 'train_samples_per_second': 49.922, 'train_steps_per_second': 3.12, 'total_flos': 6.76697418718249e+16, 'train_loss': 1.8378835439453125, 'epoch': 1.0})